In [12]:
# read configs
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))
from steps import conf

data_url = "https://liftingcast.com/meets/mvcjgjxtrlxi/results"

In [3]:
import polars as pl

In [62]:
reference_tables = "../reference-tables/csv/usa-power-lifting-australia-tsf-team-champs-at-the-fort-2024.opl.csv"

df = pl.read_csv(reference_tables, skip_rows=5)

# Rename
renamed_columns = {column: conf.camel_to_snake(column) for column in df.columns}
team_champ_df = df.rename(mapping=renamed_columns)

# Filter for competitors
team_champ_competitors = team_champ_df.select(pl.col("name"))
team_champs_df = team_champ_df.select(["name", "birth_year", "weight_class_kg", "sex"])

In [33]:
raw_file_path = "../data/raw/openpowerlifting-latest.parquet"
raw_df = pl.read_parquet(raw_file_path)

In [57]:
team_champ_competitors_best_lifts = raw_df.filter(pl.col("name").is_in(team_champ_competitors)).group_by(["name", "weight_class_kg"]).agg(pl.max("total_kg"))

In [84]:
team_champs_df.join(
    team_champ_competitors_best_lifts,
    on=["name", "weight_class_kg"],
    how="left",
).with_columns(
    rn=pl.col("total_kg").rank("ordinal", descending=True).over(["weight_class_kg", "sex"])
).sort(by=pl.col("rn")).partition_by(["weight_class_kg", "sex"])

[shape: (4, 6)
 ┌─────────────────┬────────────┬─────────────────┬─────┬──────────┬──────┐
 │ name            ┆ birth_year ┆ weight_class_kg ┆ sex ┆ total_kg ┆ rn   │
 │ ---             ┆ ---        ┆ ---             ┆ --- ┆ ---      ┆ ---  │
 │ str             ┆ str        ┆ str             ┆ str ┆ f64      ┆ u32  │
 ╞═════════════════╪════════════╪═════════════════╪═════╪══════════╪══════╡
 │ Carol Chew      ┆ 1978       ┆ 56              ┆ F   ┆ null     ┆ null │
 │ Thuy Vy Pham    ┆ 2000       ┆ 56              ┆ F   ┆ null     ┆ null │
 │ Claudia Coppola ┆ 1995       ┆ 56              ┆ F   ┆ null     ┆ null │
 │ Lina Tran       ┆ 1999       ┆ 56              ┆ F   ┆ 356.5    ┆ 1    │
 └─────────────────┴────────────┴─────────────────┴─────┴──────────┴──────┘,
 shape: (10, 6)
 ┌─────────────────────┬────────────┬─────────────────┬─────┬──────────┬──────┐
 │ name                ┆ birth_year ┆ weight_class_kg ┆ sex ┆ total_kg ┆ rn   │
 │ ---                 ┆ ---        ┆ ---       